In [1]:
dir = 'KG82sd.d'
dir = 'l22x6.d'
dir = 'l22x6pd.d'

def filename(basename):
    return dir + '/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [2]:
import pandas as pd
import math

In [3]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.sumFX = 0.
        self.sumFY = 0.
        self.sumMZ = 0.

In [4]:
table = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in table.iterrows():
    if n.ID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.ID))
    NODES[n.ID] = Node(n.ID,float(n.X),float(n.Y))

In [5]:
class Member(object):
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L

In [6]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.ID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.ID))
    MEMBERS[m.ID] = Member(m.ID,NODES[m.NODEJ],NODES[m.NODEK])

In [7]:
table = pd.read_csv(filename('node_loads'))
for i,p in table.iterrows():
    dirn = p.DIRN.upper()
    if dirn in ['FX','FY','MZ']:
        n = NODES[p.ID]
        a = 'sum'+dirn
        setattr(n,a,getattr(n,a,0.)+float(p.F))
    else:
        Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))

In [8]:
try:
    table = pd.read_csv(filename('pdelta_forces'))
    for i,p in table.iterrows():
        dirn = p.DIRN.upper()
        if dirn in ['FX','FY','MZ']:
            n = NODES[p.ID]
            a = 'sum'+dirn
            setattr(n,a,getattr(n,a,0.)+float(p.F))
        else:
            Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))
except OSError:
    pass

In [9]:
table = pd.read_csv(filename('reactions'))
for i,r in table.iterrows():
    n = NODES[r.ID]
    n.sumFX += 0. if pd.isnull(r.FX) else float(r.FX)
    n.sumFY += 0. if pd.isnull(r.FY) else float(r.FY)
    n.sumMZ += 0. if pd.isnull(r.MZ) else float(r.MZ)

In [10]:
mtable = pd.read_csv(filename('mefs'))
for i,row in mtable.iterrows():
    m = MEMBERS[row.ID]
    n = m.nodej
    n.sumFX -= row.FXJ*m.cosx - row.FYJ*m.cosy
    n.sumFY -= row.FXJ*m.cosy + row.FYJ*m.cosx
    n.sumMZ -= row.MZJ
    n = m.nodek
    n.sumFX -= row.FXK*m.cosx - row.FYK*m.cosy
    n.sumFY -= row.FXK*m.cosy + row.FYK*m.cosx
    n.sumMZ -= row.MZK
mtable

,ID,FXJ,FYJ,MZJ,FXK,FYK,MZK
0,CA0A1,542068.693843,944801.389260,3.345741e+09,-542068.693843,-944801.389260,2.795468e+09
1,CB0B1,16222731.306200,995542.969519,3.458844e+09,-16222731.306200,-995542.969519,3.012185e+09
2,CC0C1,3631365.956530,930121.493962,3.322237e+09,-3631365.956530,-930121.493962,2.723553e+09
3,CD0D1,11395838.401000,278203.458179,4.139365e-07,-11395838.401000,-278203.458179,1.808322e+09
4,CE0E1,17420795.642500,218004.089744,-2.066081e-07,-17420795.642500,-218004.089744,1.417027e+09
5,CF0F1,3284831.881220,964089.569784,3.399037e+09,-3284831.881220,-964089.569784,2.867545e+09
6,CG0G1,13479968.118800,1009122.441770,3.494127e+09,-13479968.118800,-1009122.441770,3.065168e+09
7,CA1A2,1135902.326850,531779.543827,1.417337e+09,-1135902.326850,-531779.543827,1.507450e+09
8,CB1B2,14860097.673100,646815.002045,1.744463e+09,-14860097.673100,-646815.002045,1.813020e+09
9,CC1C2,3839216.130230,501695.281296,1.289995e+09,-3839216.130230,-501695.281296,1.469329e+09


In [11]:
sums = pd.DataFrame([(n.id,n.sumFX,n.sumFY,n.sumMZ) for n in NODES.values()],
                    columns=['ID','sumFX','sumFY','sumMZ']).set_index(['ID'])
sums

,sumFX,sumFY,sumMZ
ID,,,
B13,-4.999893e-07,2.999848e-06,-3.000140e-03
C4,2.501474e-08,1.000182e-06,0.000000e+00
G15,1.299522e-07,-7.000228e-06,-5.000114e-03
B1,0.000000e+00,8.999929e-05,0.000000e+00
G6,-2.999695e-07,-4.999922e-06,-1.000023e-02
F3,-3.307468e-08,-6.000104e-06,0.000000e+00
B16,7.600029e-07,2.000190e-06,0.000000e+00
B14,6.999908e-07,-2.999848e-06,0.000000e+00
G0,-4.999922e-06,0.000000e+00,0.000000e+00


In [12]:
sums.abs().max()

sumFX    0.000005
sumFY    0.000090
sumMZ    0.010000
dtype: float64

In [13]:
from numpy.linalg import norm

In [14]:
norm(sums.sumFX,2)/min(norm(mtable.FXJ,2),norm(mtable.FXK,2))

1.3568661340443707e-13

In [15]:
norm(sums.sumFY,2)/min(norm(mtable.FYJ,2),norm(mtable.FYK,2))

4.5163385004062489e-11

In [16]:
norm(sums.sumMZ,2)/min(norm(mtable.MZJ,2),norm(mtable.MZK,2))

2.2066306344401101e-12

In [26]:
norm(mtable.FXJ,2),norm(mtable.FXK,2),norm(mtable.FXJ,2)/norm(mtable.FXK,2)

(78319597.650789976, 78319597.650789976, 1.0)

In [27]:
norm(mtable.FYJ,2),norm(mtable.FYK,2),norm(mtable.FYJ,2)/norm(mtable.FYK,2)

(5743697.1669340841, 7754792.3419932686, 0.74066421299654572)

In [25]:
norm(mtable.MZJ,2),norm(mtable.MZK,2), norm(mtable.MZJ,2)/norm(mtable.MZK,2)

(21496408161.780849, 24868315486.750706, 0.86440950024273522)